# Import library

In [1]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from onlineTripletloss import *
from selector import *
from model_SAGAN1_1 import NetG, NetD, NetA
# from model_SAGAN2_Triplet import NetG, NetD, NetA
# from model_WGANGP import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_siGAN import NetG, NetD, NetA
from dataset2Loader import CASIABDataset
# from dataset2Loader_newtriplet import CASIABDataset
# from dataset2Loader_triplet import CASIABDataset
import torch.optim as optim
import visdom
from torchvision.utils import make_grid
# Data_Dir = '../GaitRecognition/DatasetB_GEI_64x64_allseq/'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_64x64_SAGAN_90_trial16'
Model_dir = './Transform_Model/'+ Model_Name
if not os.path.isdir(Model_dir):
    os.mkdir(Model_dir)

In [ ]:

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
import os

for r, d, files in os.walk(Data_Dir):
    print(r)
    print(len(d))
    print(len(files))
    
# cpt = sum([len(files) for r, d, files in os.walk(Data_Dir)])
# cpt = sum([len(d) for r, d, files in os.walk(Data_Dir)])
# print(cpt)
# list = os.listdir(Data_Dir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# import fnmatch
# print(len(fnmatch.filter(os.listdir(Data_Dir), '*.png')))

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(16)

## iteration 刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.xavier_normal_(tensor, gain=1.)
#         init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print('Training starts')
while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
    label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, label)
    lossD += lossD_real1.item()
    lossD_real1.backward()

    label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, label)
    lossD += lossD_real2.item()
    lossD_real2.backward()

    fake = netg(img).detach()
    label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, label)
    lossD += lossD_fake.item()
    lossD_fake.backward()

    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

    label.fill_(real_label)
    output1 = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output1, label)
    lossA += lossA_real1.item()
    lossA_real1.backward()

    label.fill_(fake_label)
    output = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output, label)
    lossA += lossA_real2.item()
    lossA_real2.backward()

    label.fill_(fake_label)
    output = neta(faked)
    lossA_fake = F.binary_cross_entropy(output, label)
    lossA += lossA_fake.item()
    lossA_fake.backward()
    
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)

    label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, label)
    lossG += lossGD.item()
    lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
    label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, label)
    lossG += lossGA.item()
    lossGA.backward()
    
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 25
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 1000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

    if iteration % 100==0 or iteration==10 :
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')
            
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))


## Epoch

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
train_loader = th.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=False)
train_loader.next()

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            fake = netg(img).detach()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        



## update k times

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 2
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))    
        

## model changed +Dp

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netp = NetP(nc=1)
neta = NetA(nc=1)
netd = NetD(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netp.children())[0].children(),
    list(neta.children())[0].children(),
    list(netd.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netp = netp.to(device)
neta = neta.to(device)
netd = netd.to(device)
netg.train()
netp.train()
neta.train()
netd.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimP = optim.Adam(netp.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        
        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()


        # update P
        lossP = 0
        optimP.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossP_real1 = F.binary_cross_entropy(output1, label)
        lossP += lossP_real1.item()/2
        lossP_real1 = lossP_real1/2
        lossP_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossP_real2 = F.binary_cross_entropy(output, label)
        lossP += lossP_real2.item()/2
        lossP_real2 = lossP_real2/2
        lossP_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossP_fake = F.binary_cross_entropy(output, label)
        lossP += lossP_fake.item()/2
        lossP_fake = lossP_fake/2
        lossP_fake.backward()

        optimP.step()
    

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        
        fake = netg(img)
        output = netd(fake)
        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward(retain_graph=True)
        
        faked = th.cat((img, fake), 1)
        output = netp(faked)
        label.fill_(real_label)
        lossGP = F.binary_cross_entropy(output, label)
        lossG += lossGP.item()/2
        lossGP = lossGP/2
        lossGP.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch, epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD', 'lossP']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch, epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}  \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        



## model only D_A

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
# netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
# optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
#             # update D
#             lossD = 0
#             optimD.zero_grad()
#             output = netd(ass_label)
#             label.fill_(real_label)
#             lossD_real1 = F.binary_cross_entropy(output, label)
#             lossD += lossD_real1.item()
#             lossD_real1.backward()

#             label.fill_(real_label)
#             output1 = netd(noass_label)
#             lossD_real2 = F.binary_cross_entropy(output1, label)
#             lossD += lossD_real2.item()
#             lossD_real2.backward()

#             fake = netg(img).detach()
#             label.fill_(fake_label)
#             output2 = netd(fake)
#             lossD_fake = F.binary_cross_entropy(output2, label)
#             lossD += lossD_fake.item()
#             lossD_fake.backward()

#             optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        ]]),
                           Y=np.array([[lossG/2, lossA/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  ]]),
                     Y=np.array([[lossG/2, lossA/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {} \n'.format(
            epoch, lossG/2, lossA/3,
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}'.format(
            epoch, lossG/2, lossA/3, 
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}\n'.format(
            epoch, lossG/2, lossA/3
        ))    
        

# GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '036'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 5
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach())
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked.detach())
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            # constrain on generator
            fake_ass, P = netg(ass_label)
            fake_noass, N = netg(noass_label)
            lossTriplet = F.triplet_margin_loss(fake, fake_ass, fake_noass, margin = margin)
            lossG += lossTriplet.item()
            lossTriplet.backward()

            # constrain on encoder
    #         __, P = netg(ass_label)
    #         __, N = netg(noass_label)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# GaitGAN k times and triplet   

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 10
n_g = 2
n_d = 2


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/3)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
        
#         if(i ==0):
#             print(label_neg,label_anc,label_pos)
#             print(com_label)
#         print("shape",ass_label.shape,noass_label.shape,img.shape,com_img.shape, com_label.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach()) #需要 detach 因為不希望更新fake的參數
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)  ##這裡需要retain graph 因為他之後有需要fake，因此需要retain

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            ## new tripletloss
            _, P = netg(ass_img)
            __, N = netg(noass_img)
            lossf = TripletLoss(margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
            lossTriplet =lossf(A, P, N)
            lossG += lossTriplet.item()
            lossTriplet.backward()

    #         ## new onlinetripletloss
    #         __, com = netg(com_img)
    #         loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
    #         lossTriplet,len_triplet = loss_fn(com, com_label)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         print(lossTriplet.item(),len_triplet)

    #         ## triplet loss
    #         __, P = netg(ass_img)
    #         __, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         if i%10==0:
    # #             print("tripletloss ",lossTriplet.item())

            ## tripletloss no negative
    #         N_plus = th.zeros((A.size()), requires_grad=False).to(device)
    #         lossTriplet_AP = F.triplet_margin_loss(A, P, N_plus, margin = margin)
    #         lossG += lossTriplet_AP.item()
    #         lossTriplet += lossTriplet_AP
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG  Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# from PixelDT code刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

real_label = th.zeros((batchSize, 1), requires_grad=False).to(device)
fake_label = th.ones((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr)
optimD = optim.Adam(netd.parameters(), lr=lr)
optimA = optim.Adam(neta.parameters(), lr=lr)

print('Training starts')
while iteration < 40000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
#     label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, real_label)
#     lossD += lossD_real1.item()
#     lossD_real1.backward()

#     label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, real_label)
#     lossD += lossD_real2.item()
#     lossD_real2.backward()

    fake = netg(img).detach()
#     label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, fake_label)
#     lossD += lossD_fake.item()
#     lossD_fake.backward()
    lossD = (lossD_real1+ lossD_real2+ lossD_fake)/3
    lossD.backward()

    lossD_item = lossD.item()
    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

#     label.fill_(real_label)
    output = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output, real_label)
#     lossA += lossA_real1.item()
#     lossA_real1.backward()

#     label.fill_(fake_label)
    output1 = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output1, fake_label)
#     lossA += lossA_real2.item()
#     lossA_real2.backward()

#     label.fill_(fake_label)
    output2 = neta(faked)
    lossA_fake = F.binary_cross_entropy(output2, fake_label)
#     lossA += lossA_fake.item()
#     lossA_fake.backward()
    lossA = (lossA_real1+ lossA_real2 +lossA_fake)/3
    lossA.backward()
    
    lossA_item = lossA.item()
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)
#     label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, real_label)
#     lossG += lossGD.item()
#     lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
#     label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, real_label)
#     lossG += lossGA.item()
#     lossGA.backward()
    lossG = (lossGD + lossGA)/2
    lossG.backward()
    
    lossG_item = lossG.item()
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 5000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
    if iteration % 5000==0 or iteration==10 or iteration==500:
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                           opts=dict(
                               title='GaitGAN',
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                     win=win,
                     update='append')


# WGAN 

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 1
n_d = 2
clip = 0.1


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.RMSprop(netg.parameters(), lr=glr/2)
optimD = optim.RMSprop(netd.parameters(), lr=dlr/3)
optimA = optim.RMSprop(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={},clip={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp, clip))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD += lossD_real1.item()
            lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            for p in netd.parameters():
                p.data.clamp_(-clip, clip)

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

            for p in neta.parameters():
                p.data.clamp_(-clip, clip)

        # update G
        if i % n_d == 0:
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
        
        


n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 3.752237558364868, ErrA = -1.8985527356465657, ErrD = -3.5465147892634072
Epoch = 4, ErrG = 5.705430269241333, ErrA = -2.807316621144613, ErrD = -5.611066023508708
Epoch = 6, ErrG = 4.896071195602417, ErrA = -3.07237180074056, ErrD = -6.601948658625285
Epoch = 8, ErrG = 8.87260103225708, ErrA = -4.403616587320964, ErrD = -9.87374464670817
Epoch = 10, ErrG = 7.947443723678589, ErrA = -4.1470723152160645, ErrD = -8.064921021461487
Epoch = 12, ErrG = 1.238088607788086, ErrA = -4.921435038248698, ErrD = -4.710442225138347
Epoch = 14, ErrG = 0.5194168090820312, ErrA = -5.6255998611450195, ErrD = -5.323735237121582
Epoch = 16, ErrG = 6.348399639129639, ErrA = -7.5277970631917315, ErrD = -7.016017913818359
Epoch = 18, ErrG = 0.8842306137084961, ErrA = -7.25491460164388, ErrD = -7.062686284383138
Epoch = 20, ErrG = 0.0035724639892578125, ErrA = -7.803771336873372, ErrD = -7.927731831868489
Epoch = 22, E

 # WGAN-GP

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.999
margin = 0
n_g = 0
n_d = 5

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD_ += lossD_real1
            lossD += lossD_real1.item()
    #         lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD_ += lossD_real2
            lossD += lossD_real2.item()
    #         lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD_ += lossD_fake
            lossD += lossD_fake.item()
            gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
    #         lossD_fake.backward()
            lossD_ = lossD_/3 + lambda_gp * gradient_penalty
            lossD_.backward()

            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_ += lossA_real1
    #         lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_ += lossA_real2
    #         lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_ += lossA_fake
    #         lossA_fake.backward()
            gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3 + lambda_gp * gradient_penalty

            lossA_.backward()
            optimA.step()
    
            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossG_ += lossGD
#             lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossG_ += lossGA
#             lossGA.backward()
            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        
        


# SA GaitGAN (hing)

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 0
n_g = 1
n_d = 2


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, gf1 = netg(img)
            d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, gf1 = netg(img)
            faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake,_= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
            #如果爾後有要再backward，第一次就需要retain graph
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 1.616885781288147, ErrA = 0.45645807186762494, ErrD = 0.0, Gattn=-0.0020963058341294527, Dattn=0.0011602682061493397, Aattn=-0.001853231806308031
Epoch = 4, ErrG = 0.8820965886116028, ErrA = 0.3970751961072286, ErrD = 0.17267785842219988, Gattn=-0.00410621939226985, Dattn=0.0021039096172899008, Aattn=-0.002656357828527689
Epoch = 6, ErrG = 1.8386868834495544, ErrA = 0.494064340988795, ErrD = 0.13149824738502502, Gattn=-0.00606911163777113, Dattn=0.006608845666050911, Aattn=-0.003808614332228899
Epoch = 8, ErrG = 0.9222525954246521, ErrA = 0.6054384907086691, ErrD = 0.3740019624431928, Gattn=-0.008033267222344875, Dattn=0.014804177917540073, Aattn=-0.00481810700148344
Epoch = 10, ErrG = 1.367496907711029, ErrA = 0.5269833107789358, ErrD = 0.290728231271108, Gattn=-0.009626723825931549, Dattn=0.0232836883515119

Epoch = 94, ErrG = 1.6275362968444824, ErrA = 0.12814308951298395, ErrD = 0.0, Gattn=-0.016189202666282654, Dattn=0.07154933363199234, Aattn=-0.07755158096551895
Epoch = 96, ErrG = 1.6325467824935913, ErrA = 0.13576739033063254, ErrD = 0.0, Gattn=-0.01624268665909767, Dattn=0.0722448006272316, Aattn=-0.07858452945947647
Epoch = 98, ErrG = 1.6740132570266724, ErrA = 0.14139717817306519, ErrD = 0.023082298537095387, Gattn=-0.01623196341097355, Dattn=0.07272286713123322, Aattn=-0.07957756519317627
Epoch = 100, ErrG = 1.3588250875473022, ErrA = 0.20691749826073647, ErrD = 0.009358682359258333, Gattn=-0.01625732146203518, Dattn=0.07334950566291809, Aattn=-0.08057785034179688
Epoch = 102, ErrG = 2.0385136008262634, ErrA = 0.2686113491654396, ErrD = 0.01747746703525384, Gattn=-0.016284307464957237, Dattn=0.07384389638900757, Aattn=-0.08147003501653671
Epoch = 104, ErrG = 1.8831560611724854, ErrA = 0.18347068627675375, ErrD = 0.047291224201520286, Gattn=-0.016303449869155884, Dattn=0.074405699

Epoch = 188, ErrG = 1.7993968725204468, ErrA = 0.12923596799373627, ErrD = 0.0, Gattn=-0.01757741905748844, Dattn=0.09618420153856277, Aattn=-0.12102954834699631
Epoch = 190, ErrG = 1.8903300166130066, ErrA = 0.13277608528733253, ErrD = 0.0, Gattn=-0.017606057226657867, Dattn=0.09650719165802002, Aattn=-0.12202966213226318
Epoch = 192, ErrG = 1.5198928117752075, ErrA = 0.14892739554246268, ErrD = 0.0, Gattn=-0.01762796752154827, Dattn=0.09716184437274933, Aattn=-0.12291866540908813
Epoch = 194, ErrG = 1.6910642981529236, ErrA = 0.17905878772338232, ErrD = 0.0, Gattn=-0.017646370455622673, Dattn=0.09762880951166153, Aattn=-0.12369631975889206
Epoch = 196, ErrG = 1.9219459891319275, ErrA = 0.07201228787501653, ErrD = 0.0765681800742944, Gattn=-0.017680013552308083, Dattn=0.09814928472042084, Aattn=-0.12495751678943634
Epoch = 198, ErrG = 1.6486031413078308, ErrA = 0.1653051475683848, ErrD = 0.015836510807275772, Gattn=-0.017693473026156425, Dattn=0.0984291359782219, Aattn=-0.125680416822

Epoch = 290, ErrG = 3.092173457145691, ErrA = 0.03599401315053304, ErrD = 0.0, Gattn=-0.01902659796178341, Dattn=0.12828309834003448, Aattn=-0.14506329596042633
Epoch = 292, ErrG = 4.209545016288757, ErrA = 0.0005994904786348343, ErrD = 0.0, Gattn=-0.01896648295223713, Dattn=0.12828309834003448, Aattn=-0.14552220702171326
Epoch = 294, ErrG = 3.1149765253067017, ErrA = 0.0, ErrD = 0.0, Gattn=-0.018983880057930946, Dattn=0.12828309834003448, Aattn=-0.14589282870292664
Epoch = 296, ErrG = 3.9159406423568726, ErrA = 0.008155032992362976, ErrD = 0.0, Gattn=-0.019002065062522888, Dattn=0.12828309834003448, Aattn=-0.14655770361423492
Epoch = 298, ErrG = 3.8342342376708984, ErrA = 0.004564343641201655, ErrD = 0.0, Gattn=-0.019022921100258827, Dattn=0.12828309834003448, Aattn=-0.14731480181217194
Epoch = 300, ErrG = 2.722634196281433, ErrA = 0.03775585566957792, ErrD = 0.0, Gattn=-0.01900261640548706, Dattn=0.12828309834003448, Aattn=-0.14766381680965424
Epoch = 302, ErrG = 2.7025136947631836, 

Epoch = 398, ErrG = 4.420372724533081, ErrA = 0.008453298980991045, ErrD = 0.0, Gattn=-0.019029056653380394, Dattn=0.12828309834003448, Aattn=-0.17681127786636353
Epoch = 400, ErrG = 5.574413776397705, ErrA = 0.003455959881345431, ErrD = 0.0, Gattn=-0.019121088087558746, Dattn=0.12828309834003448, Aattn=-0.17690542340278625
Epoch = 402, ErrG = 5.717231512069702, ErrA = 0.003681067998210589, ErrD = 0.0, Gattn=-0.019124507904052734, Dattn=0.12828309834003448, Aattn=-0.1768314093351364
Epoch = 404, ErrG = 4.768608331680298, ErrA = 0.0, ErrD = 0.0, Gattn=-0.01907649077475071, Dattn=0.12828309834003448, Aattn=-0.17736266553401947
Epoch = 406, ErrG = 4.5890949964523315, ErrA = 0.0336657619724671, ErrD = 0.0, Gattn=-0.019104810431599617, Dattn=0.12828309834003448, Aattn=-0.17808718979358673
Epoch = 408, ErrG = 5.096924662590027, ErrA = 0.0, ErrD = 0.0, Gattn=-0.01908019371330738, Dattn=0.12828309834003448, Aattn=-0.17855317890644073
Epoch = 410, ErrG = 4.457731008529663, ErrA = 0.0, ErrD = 0.

KeyboardInterrupt: 

# SA GaitGAN (WGAN-GP) 刪

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        
        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = -th.mean(d_out_assreal)

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = -th.mean(d_out_noassreal)

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = th.mean(d_out_fake)

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
#         lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = -th.mean(d_out_assreal)
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = th.mean(d_out_noassreal)

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = th.mean(d_out_faked)

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
#         lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()

            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# SA GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 0

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            # constrain on generator
            fake_ass, P = netg(ass_img)
            fake_noass, N = netg(noass_img)
            lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
            lossG_ += lossTriplet
            lossG += lossTriplet.item()
    #         lossTriplet.backward()

            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

# SA GaitGAN and triplet (onlineloss)

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 0

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img, label_neg, label_anc, label_pos) in enumerate(train_loader):
        
        com_img = th.cat((noass_img, img, ass_img), 0)
        com_label = th.cat(( label_neg, label_anc, label_pos), 0)
        com_img = com_img.to(device).to(th.float32)
        com_label = com_label.to(device).to(th.float32)
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()


            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

    #         # constrain on encoder
    #         fake_ass, P = netg(ass_img)
    #         fake_noass, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG_ += lossTriplet
    #         lossG += lossTriplet.item()
    # #         lossTriplet.backward()

            ## new onlinetripletloss
            __, com = netg(com_img)
            loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
            lossTriplet,len_triplet = loss_fn(com, com_label)
            lossG += lossTriplet.item()
            lossG_ += lossTriplet
    #         lossTriplet.backward()


            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           